A Convolutional Network implementation example using TensorFlow library.

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### Parameters

In [3]:
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 20

### Network Parameters

In [4]:
n_input = 784 # MNIST data input (img_shape = 28*28)
n_classes = 10 # MNIST total classes (0-9)
dropout = 0.75 # Dropout, probability to keep units

### tf Graph input

In [5]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

### Create Model

In [6]:
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, 
                                                  w, 
                                                  strides=[1,1,1,1], 
                                                  padding='SAME'), 
                                     b))

In [7]:
def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [13]:
def conv_net(_X, _weights, _biases, _dropout):
    #Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])
    
    # Convolution layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply dropout
    conv1 = tf.nn.dropout(conv1, _dropout)
    
    # Convolution layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply dropout
    conv2 = tf.nn.dropout(conv2, _dropout)
    
    # Fully connected layer
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]])
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply dropout
    dense1 = tf.nn.dropout(dense1, _dropout)
    
    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

Store weights and biases

In [14]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

In [15]:
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([10]))
}

Construct model

In [16]:
pred = conv_net(x, weights, biases, keep_prob)

Define loss and optimizer

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Evaluate the model

In [18]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Initialize the variables

In [19]:
init = tf.initialize_all_variables()

Launch the graph

In [24]:
with tf.Session() as sess:
    sess.run(init)
    
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            
            print("Iter {}, Minibatch loss = {:.6f}, Training Accuracy = {:.5f}".format(step * batch_size, 
                                                                                        loss, 
                                                                                        acc))
        
        step += 1
        
    print("Optimization finished!")
    # Calculate accuracy for 256 mnist test images
    test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                             y: mnist.test.labels[:256],
                                             keep_prob: 1.})
    
    print("Testing accuracy: {}".format(test_acc))

Iter 2560, Minibatch loss = 14724.944336, Training Accuracy = 0.29688
Iter 5120, Minibatch loss = 8007.062500, Training Accuracy = 0.63281


KeyboardInterrupt: 